In [1]:
import os

import numpy as np
import pandas as pd
import re
import paramiko
from getpass import getpass
import traceback
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

/home/stotoshka/Soft/anaconda3/envs/research/lib/python3.9/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
os.chdir("/home/stotoshka/Documents/ANGEL_dev")


In [3]:
%%bash
cd /home/stotoshka/Documents/ANGEL_dev
python Proteasome/in_vitro/prepareValidation.py -t I -w 2

/home/stotoshka/Documents/ANGEL_dev/Proteasome/in_vitro/prepareValidation.py:53: FutureWarning: The default value of regex will change from True to False in a future version.
  calis_data_filtered["sequence"] = calis_data_filtered["cleavage_sites"].str.replace("\\|","")
/home/stotoshka/Documents/ANGEL_dev/Proteasome/in_vitro/prepareValidation.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calis_data_filtered["sequence"] = calis_data_filtered["cleavage_sites"].str.replace("\\|","")


0    753
1    389
Name: activity, dtype: int64
Num train substrate = 24
Num validation substrate = 215
0    1560
1     737
Name: activity, dtype: int64


In [4]:
%%time
path_to_config = "Proteasome/in_vitro/immuno/converter_config.json"
command = f"python tools/generateConfigForConverter.py -i {os.path.join(os.getcwd(),'Proteasome/in_vitro/immuno/validation/ready_train_immuno_2.csv')} -o {os.path.join(os.getcwd(),'Proteasome/in_vitro/immuno/sdf')} -c peptide -t 10 {path_to_config}"
os.system(command)
print("Convertation")
convert_command = f"python tools/SeqToSDF.py {path_to_config}"
os.system(convert_command)

Success
Convertation
Success
--- 0.17413783073425293 seconds ---
CPU times: user 901 µs, sys: 3.39 ms, total: 4.29 ms
Wall time: 610 ms


0

In [5]:
%%time
path_to_config = "Proteasome/in_vitro/immuno/converter_config.json"
command = f"python tools/generateConfigForConverter.py -i {os.path.join(os.getcwd(),'Proteasome/in_vitro/immuno/validation/ready_validation_immuno_2.csv')} -o {os.path.join(os.getcwd(),'Proteasome/in_vitro/immuno/sdf')} -c peptide -t 10 {path_to_config}"
os.system(command)
print("Convertation")
convert_command = f"python tools/SeqToSDF.py {path_to_config}"
os.system(convert_command)

Success
Convertation
Success
--- 0.3956427574157715 seconds ---
CPU times: user 3.16 ms, sys: 408 µs, total: 3.56 ms
Wall time: 725 ms


0

In [8]:
%%bash
cd /home/stotoshka/Documents/ANGEL_dev
rm Proteasome/in_vitro/immuno/sdf/*.txt
cp Proteasome/in_vitro/immuno/sdf/ready_train_immuno_2.sdf /run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/sdf
cp Proteasome/in_vitro/immuno/sdf/ready_validation_immuno_2.sdf /run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/sdf

In [9]:
command = f"python tools/generateConfigForTrain.py -l {7} -b total_immuno_2_7 -s sdf/ready_train_immuno_2.sdf -a activity -p models/total_immuno_2_7 /run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/total_immuno_2_7_tr.txt"
os.system(command)

Success


0

In [11]:
command = f"python tools/generateConfigForPrediction.py -m models/total_immuno_2_7.MSAR -s sdf/ready_validation_immuno_2.sdf -a sequence -p results /run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/total_immuno_2_7_val.txt"
os.system(command)

Success


0

In [12]:
def checkTrain(path):
    flag = False
    header = "No	 Check	 Number	 IAP	 20-Fold	 Activity Type"
    try:
        with open(path, "r", encoding="cp1252") as log:
            for line in log:
                if header in line:
                    flag = True
    except:
        traceback.print_exc()
    finally:
        return flag


def checkValidation(path):
    try:
        f = False
        with open(path, "r", encoding="cp1252") as log:
            for line in log:
                if "are predicted" in line:
                    comps = line.strip().split(" ")
                    if int(comps[0]) == int(comps[2]):
                        f = True
                    else:
                        f = False
    except:
        traceback.print_exc()
    finally:
        return f

In [13]:
%%time
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
password = getpass()
client.connect('192.168.0.10', username='SmirnygaTotoshka', password=password)

tr_command = r"C:\Users\SmirnygaTotoshka\Desktop\OLMPASS\OLMPASSdoSAR.exe C:\Users\SmirnygaTotoshka\Desktop\diplom\ANGEL\immuno\total_immuno_2_7_tr.txt"
stdin, stdout, stderr = client.exec_command(tr_command)  # Non-blocking call
exit_status = stdout.channel.recv_exit_status()  # Blocking call
if exit_status == 0:
    if checkTrain(f"/run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/models/total_immuno_2_7.HST"):
        print("Train ended")
    else:
        print(f"Train ended with errors")
else:
    print("Error", exit_status)
client.close()

Train ended
CPU times: user 71.5 ms, sys: 17.7 ms, total: 89.1 ms
Wall time: 36.2 s


In [14]:
%%time
client.connect('192.168.0.10', username='SmirnygaTotoshka', password=password)
val_command = r"C:\Users\SmirnygaTotoshka\Desktop\OLMPASS\OLMPASS2CSV.exe C:\Users\SmirnygaTotoshka\Desktop\diplom\ANGEL\immuno\total_immuno_2_7_val.txt"
stdin_val, stdout_val, stderr_val = client.exec_command(val_command)  # Non-blocking call
exit_status_val = stdout_val.channel.recv_exit_status()
if exit_status_val == 0:
    if checkValidation(f"/run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/results/ready_validation_immuno_2-total_immuno_2_7.HST"):
        print("Validation ended")
    else:
        print(f"Validation ended with errors")
else:
    print("Error validation", exit_status_val)
client.close()

Validation ended with errors
CPU times: user 41.1 ms, sys: 7.62 ms, total: 48.7 ms
Wall time: 10.9 s


In [16]:
%%bash
cd /home/stotoshka/Documents/ANGEL_dev
cp /run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/models/total* Proteasome/in_vitro/immuno/results/validation/models/
cp /run/user/1000/gvfs/smb-share:server=192.168.0.10,share=diplom/ANGEL/immuno/results/ready_validation* Proteasome/in_vitro/immuno/results/validation/results

In [17]:
path = f"Proteasome/in_vitro/immuno/results/validation/models/total_immuno_2_7.HST"
header = "No	 Check	 Number	 IAP	 20-Fold	 Activity"
tbl = pd.DataFrame(columns=["model_name","window", "descriptor_level", "iap", "twentyCV", "activity", "num_subst"])
with open(path, "r", encoding='cp1252') as f:
    split_name = re.split("_", os.path.splitext(os.path.basename(path))[0])
    model_name = split_name[1]
    window = split_name[2]
    level = split_name[3]
    lines = f.readlines()
    flag = False
    for line in lines:
        if header in line:
            flag = True
            continue
        if line == "\n" and flag:
            break
        if flag:
            components = re.split("\t\\s+", line)
            num_subst = components[2]
            iap = components[3]
            twentyCV = components[4]
            activity = components[5].strip()
            row = pd.DataFrame.from_dict({"model_name": [model_name],
                                          "window":[int(window)],
                                          "descriptor_level": [int(level)],
                                          "num_subst": [int(num_subst)],
                                          "iap": [float(iap.replace(",", "."))],
                                          "twentyCV": [float(twentyCV.replace(",", "."))],
                                          "activity": [int(activity)]})
            tbl = pd.concat([tbl, row], ignore_index=True)
    if not flag:
        print(f"total not any predictable activity")
tbl.to_excel("Proteasome/in_vitro/immuno/results/validation/internal.xlsx", index = False)

In [18]:
internal_res = pd.read_excel("Proteasome/in_vitro/immuno/results/validation/internal.xlsx")
internal_res.head()

,model_name,window,descriptor_level,iap,twentyCV,activity,num_subst
0,immuno,2,7,0.6392,0.6347,1,350
1,immuno,2,7,0.6469,0.6426,0,717


In [19]:
def deconvoluteTrue(X, excluded=None):
    seqs = list(X["substrate"].unique())
    for i in X.index:
        finded_start = X.loc[i, "substrate"].find(X.loc[i, "fragment"])
        X.loc[i, "start"] = finded_start - 1  # сайт левее найденной позиции (левее N-конца)
        X.loc[i, "end"] = finded_start + len(X.loc[i, "fragment"]) - 1  # сайт левее найденной позиции (C-конец)
    result = pd.DataFrame(columns=["residue", "sequence", "activity"])
    k = 0
    for s in seqs:
        subset = X.query("substrate == @s").reset_index(drop=True)
        start = list(subset['start'])
        end = list(subset['end'])
        for i in range(0, len(s)):
            if excluded is not None:
                if not pd.isna(subset.loc[0,excluded]):
                    ex = subset.loc[0, excluded].split(";")
                else:
                    ex = []
                if i in ex:
                    result.loc[k] = [s[i], s, 0]
                else:
                    result.loc[k] = [s[i], s, int(i in start or i in end)]
            else:
                result.loc[k] = [s[i], s, int(i in start or i in end)]
            k += 1
    print(result.activity.value_counts())
    return result

In [20]:
sample = pd.read_csv("Proteasome/in_vitro/immuno/validation/ready_validation_immuno_2.csv",sep = ";")
sample.head()

,peptide,sequence,full_sequence,activity
0,ALETG,ALETG,ALETGTIFLPVRHD,0
1,LETGT,LETGT,ALETGTIFLPVRHD,1
2,ETGTI,ETGTI,ALETGTIFLPVRHD,0
3,TGTIF,TGTIF,ALETGTIFLPVRHD,0
4,GTIFL,GTIFL,ALETGTIFLPVRHD,0


In [21]:
prediction = pd.read_csv("Proteasome/in_vitro/immuno/results/validation/results/ready_validation_immuno_2-total_immuno_2_7.csv",sep = ";", header = 0,decimal=",")
prediction = prediction.drop(columns=["Substructure Descriptors", "New Descriptors", "Possible Activities at Pa>Pi"])
prediction.head()

,sequence,1,0
0,ALETG,-0.8602,0.8234
1,LETGT,0.0923,-0.2207
2,ETGTI,0.5642,-0.1767
3,TGTIF,-0.6327,0.5601
4,GTIFL,0.6356,-0.6448


In [22]:
j = sample.index[0]
flag = True
for i in sample.index:
    if flag:
        j = i
    else:
        j += 1
    if sample.loc[i,"peptide"] == prediction.loc[j,"sequence"]:
        sample.loc[i,"pred"] = prediction.loc[j,"1"]
        sample.loc[i,"pred_pep"] = prediction.loc[j,"sequence"]
    else:
        flag = False
        print(f"Mismatch {i} {sample.loc[i,'peptide']} {j} {prediction.loc[j,'sequence']}")
        sample.loc[i,"pred"] = 0

sample

,peptide,sequence,full_sequence,activity,pred,pred_pep
0,ALETG,ALETG,ALETGTIFLPVRHD,0,-0.8602,ALETG
1,LETGT,LETGT,ALETGTIFLPVRHD,1,0.0923,LETGT
2,ETGTI,ETGTI,ALETGTIFLPVRHD,0,0.5642,ETGTI
3,TGTIF,TGTIF,ALETGTIFLPVRHD,0,-0.6327,TGTIF
4,GTIFL,GTIFL,ALETGTIFLPVRHD,0,0.6356,GTIFL
...,...,...,...,...,...,...
2292,TMLNT,TMLNT,PEVIPMFSALSEGATPQDLNTMLNTVGGH,1,0.5622,TMLNT
2293,MLNTV,MLNTV,PEVIPMFSALSEGATPQDLNTMLNTVGGH,0,0.7821,MLNTV
2294,LNTVG,LNTVG,PEVIPMFSALSEGATPQDLNTMLNTVGGH,0,-0.4036,LNTVG
2295,NTVGG,NTVGG,PEVIPMFSALSEGATPQDLNTMLNTVGGH,0,-0.7940,NTVGG


In [23]:
window = 2
result = pd.DataFrame(columns=["residue","full_seq", "peptide","prediction"])
unique_fulls = sample.full_sequence.unique()
k = 0
for seq in unique_fulls:
    subset = sample.query("full_sequence == @seq")
    sub_i = 0
    for i,r in enumerate(seq):
        if i < window or i > len(seq)-window-1:
            result.loc[k] = [r,seq,"",0]
            k+=1
        else:
            pos = subset.index[sub_i]
            result.loc[k] = [r,seq,subset.loc[pos,"peptide"],subset.loc[pos,"pred"]]
            k+=1
            sub_i+=1

result

,residue,full_seq,peptide,prediction
0,A,ALETGTIFLPVRHD,,0.0000
1,L,ALETGTIFLPVRHD,,0.0000
2,E,ALETGTIFLPVRHD,ALETG,-0.8602
3,T,ALETGTIFLPVRHD,LETGT,0.0923
4,G,ALETGTIFLPVRHD,ETGTI,0.5642
...,...,...,...,...
3152,T,PEVIPMFSALSEGATPQDLNTMLNTVGGH,LNTVG,-0.4036
3153,V,PEVIPMFSALSEGATPQDLNTMLNTVGGH,NTVGG,-0.7940
3154,G,PEVIPMFSALSEGATPQDLNTMLNTVGGH,TVGGH,-0.8650
3155,G,PEVIPMFSALSEGATPQDLNTMLNTVGGH,,0.0000


In [24]:
angel_result = pd.DataFrame(columns=["residue", "sequence", "true","prediction"])
total_validation = pd.read_csv("Proteasome/in_vitro/immuno/validation/total_validation_immuno.csv",sep = ";")
true = deconvoluteTrue(total_validation,"exclusions")
true

0    2306
1     851
Name: activity, dtype: int64


,residue,sequence,activity
0,A,ALETGTIFLPVRHD,0
1,L,ALETGTIFLPVRHD,0
2,E,ALETGTIFLPVRHD,0
3,T,ALETGTIFLPVRHD,1
4,G,ALETGTIFLPVRHD,0
...,...,...,...
3152,T,PEVIPMFSALSEGATPQDLNTMLNTVGGH,0
3153,V,PEVIPMFSALSEGATPQDLNTMLNTVGGH,0
3154,G,PEVIPMFSALSEGATPQDLNTMLNTVGGH,0
3155,G,PEVIPMFSALSEGATPQDLNTMLNTVGGH,0


In [25]:
true["ANGEL"] = result["prediction"]

In [26]:
roc = metrics.roc_auc_score(true["activity"],true["ANGEL"])
ap = metrics.average_precision_score(true["activity"],true["ANGEL"])
print(roc,ap)

0.5825901979508827 0.3503160511991559


In [27]:
with open("Proteasome/in_vitro/immuno/validation/val_seqs.txt","w") as txt:
    unique_fulls = sample.full_sequence.unique()
    for f in unique_fulls:
        txt.write(f"{f}\n")

In [28]:
%%bash
cd /home/stotoshka/Documents/ANGEL_dev
time python Proteasome/competitors/runPepsickle.py -t I -m in-vitro Proteasome/in_vitro/immuno/validation/val_seqs.txt Proteasome/in_vitro/immuno/results/validation/pepsickle_in_vitro.xlsx

conda run -p /home/stotoshka/Soft/anaconda3/envs/pepsickle-v0-2-1/ pepsickle -f Proteasome/in_vitro/immuno/validation/pepsickle_tmp.fasta -m in-vitro -p I --human-only >> Proteasome/in_vitro/immuno/validation/pepsickle_result.txt
Finish



real	0m3.496s
user	0m3.347s
sys	0m1.292s


In [29]:
%%bash
cd /home/stotoshka/Documents/ANGEL_dev
time python Proteasome/competitors/runPepsickle.py -t I -m in-vitro-2 Proteasome/in_vitro/immuno/validation/val_seqs.txt Proteasome/in_vitro/immuno/results/validation/pepsickle_in_vitro_2.xlsx

conda run -p /home/stotoshka/Soft/anaconda3/envs/pepsickle-v0-2-1/ pepsickle -f Proteasome/in_vitro/immuno/validation/pepsickle_tmp.fasta -m in-vitro-2 -p I --human-only >> Proteasome/in_vitro/immuno/validation/pepsickle_result.txt
Finish



real	0m3.256s
user	0m4.287s
sys	0m1.338s


In [30]:
%%bash
cd /home/stotoshka/Documents/ANGEL_dev
time python Proteasome/competitors/runPCPS.py -t I Proteasome/in_vitro/immuno/validation/val_seqs.txt Proteasome/in_vitro/immuno/results/validation

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2
3 0
3 1
3 2
4 0
4 1
4 2
5 0
5 1
5 2
6 0
6 1
6 2
7 0
7 1
7 2
8 0
8 1
8 2
9 0
9 1
9 2
10 0
10 1
10 2
11 0
11 1
11 2
12 0
12 1
12 2
13 0
13 1
13 2
14 0
14 1
14 2
15 0
15 1
15 2
16 0
16 1
16 2
17 0
17 1
17 2
18 0
18 1
18 2
19 0
19 1
19 2
20 0
20 1
20 2
21 0
21 1
21 2
22 0
22 1
22 2
23 0
23 1
23 2
24 0
24 1
24 2
25 0
25 1
25 2
26 0
26 1
26 2
27 0
27 1
27 2
28 0
28 1
28 2
29 0
29 1
29 2
30 0
30 1
30 2
31 0
31 1
31 2
32 0
32 1
32 2
33 0
33 1
33 2
34 0
34 1
34 2
35 0
35 1
35 2
36 0
36 1
36 2
37 0
37 1
37 2
38 0
38 1
38 2
39 0
39 1
39 2
40 0
40 1
40 2
41 0
41 1
41 2
42 0
42 1
42 2
43 0
43 1
43 2
44 0
44 1
44 2
45 0
45 1
45 2
46 0
46 1
46 2
47 0
47 1
47 2
48 0
48 1
48 2
49 0
49 1
49 2
50 0
50 1
50 2
51 0
51 1
51 2
52 0
52 1
52 2
53 0
53 1
53 2
54 0
54 1
54 2
55 0
55 1
55 2
56 0
56 1
56 2
57 0
57 1
57 2
58 0
58 1
58 2
59 0
59 1
59 2
60 0
60 1
60 2
61 0
61 1
61 2
62 0
62 1
62 2
63 0
63 1
63 2
64 0
64 1
64 2
65 0
65 1
65 2
66 0
66 1
66 2
67 0
67 1
67 2
68 0
68 1



real	80m19.823s
user	0m25.374s
sys	0m1.216s
